# Shapenet Rendering
- Blender script source: https://github.com/panmari/stanford-shapenet-renderer

## Semantic Segmentation
If rendering a single object, we get a mask (interpretted as a binary segmentation map) for free. However, if we want to render multiple objects, either for rendering our target object into a 3D scene or for using secondary models to produce oclusions, then we need to do something more. 

Here are some references for how we can do semantic segmentation:
- https://blender.stackexchange.com/questions/79595/change-diffuse-shader-to-emission-shader-without-affecting-shader-color
- https://blender.stackexchange.com/questions/80906/create-a-segmentation-picture-with-each-object-class-rendered-in-different-color/162746#162746
- https://blender.stackexchange.com/questions/34609/is-there-a-way-to-streamline-scripting-these-shaders-and-modifier-keyframes
- https://github.com/DIYer22/bpycv#install
 - This seems promising, but I had issues when trying to install openexr

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import json
import glob
import cv2
import imageio
import shutil

import subprocess
import numpy as np

from modules.ShapeNetHandler import ShapeNetHandler
import modules.utils

from IPython.display import Image, display

# Load ShapeNet Handler

In [3]:
shapenet_root = "/hdd/mliuzzolino/ShapeNet/data/ShapeNetCore.v2"
shapenet_handler = ShapeNetHandler(shapenet_root)

## Print available categories by name
- Can use name to condition random sampling

In [4]:
# shapenet_handler.print_categories()

## Randomly Sample a filepath

In [5]:
temp_obj_filepath, temp_synset_id, temp_instance_id, temp_obj_name = shapenet_handler.sample_obj(category_name='car')

## Output Path

In [6]:
_OUTPUT_ROOT = 'render_output'

# Render

In [82]:
_N_VIEW_ANGLES = 72
_N_MODELS_TO_RENDER = 60
_RANDOM_SCALING = False
_OVERWRITE = True
_RENDER_SCRIPT = "scripts/blender_render.py"
_TARGET_CATEGORY_LIST = []#['airplane', 'car']

## Run Blender

In [ ]:
subprocess_outputs = []
try:
    for img_i in range(_N_MODELS_TO_RENDER):
        # Randomly select target category
        target_category = ''
        if _TARGET_CATEGORY_LIST != []:
            target_category = np.random.choice(_TARGET_CATEGORY_LIST)
        
        # Sample object filepath
        obj_filepath, synset_id, instance_id, obj_name = shapenet_handler.sample_obj(category_name=target_category)
        
        # Setup outpath
        output_root = os.path.join(_OUTPUT_ROOT, f'synsetID_{synset_id}', instance_id)
        
        # Randomly select obj scaling
        random_scale = np.random.uniform(0.99, 1.15) if _RANDOM_SCALING else 1.0
        random_n_occlusions = np.random.randint(3, 7)
        
        if not os.path.exists(output_root) or _OVERWRITE:
            # Remove previous directory if overwrite
            if _OVERWRITE and os.path.exists(output_root):
                shutil.rmtree(output_root)
            
            # Run blender script
            cmd = f"blender --background --python {_RENDER_SCRIPT} -- "
            cmd += f"--output_folder {output_root} --views {_N_VIEW_ANGLES} "
            cmd += f"--scale {random_scale} --n_occlusions {random_n_occlusions} "
            cmd += f"{obj_filepath}"
            out = subprocess.check_output(cmd.split(" "), shell=False)
            subprocess_outputs.append(out)
        else:
            print("Skipping.")
        
        # Stdout
        stdout_str = f"[{img_i+1}/{_N_MODELS_TO_RENDER}] -- "
        stdout_str += f"SynsetID: {synset_id} -- "
        stdout_str += f"InstanceID: {instance_id:32s} -- "
        stdout_str += f"Name: {obj_name}"
        print(stdout_str)
        
except KeyboardInterrupt:
    try:
        # Try to cleanup directory with incomplete # renders
        if len(os.listdir(output_root)) != _N_VIEW_ANGLES:
            shutil.rmtree(output_root)
    except:
        pass
    print("\nEnding early.")

[1/60] -- SynsetID: 02808440 -- InstanceID: b19ff5aaa6a6940b412deb0d2b0f3dcd -- Name: bathtub,bathing tub,bath,tub
[2/60] -- SynsetID: 03636649 -- InstanceID: 273314626729b1e973222d877df1ecac -- Name: lamp
[3/60] -- SynsetID: 03513137 -- InstanceID: ae8a2ae90ac11bd93d4e343a1ac21b93 -- Name: helmet
[4/60] -- SynsetID: 03948459 -- InstanceID: e3f45b75b688bf33794921644e32aee6 -- Name: pistol,handgun,side arm,shooting iron
[5/60] -- SynsetID: 03046257 -- InstanceID: 637237e978d5168f9751189c905e470  -- Name: clock
[6/60] -- SynsetID: 03211117 -- InstanceID: a3ada0a8bc0d4b8392ababf87635e60c -- Name: display,video display
[7/60] -- SynsetID: 02691156 -- InstanceID: 1f672d2fd5e3f4e78026abe712c1ab05 -- Name: airplane,aeroplane,plane
[8/60] -- SynsetID: 02871439 -- InstanceID: 61ebec53da40801f99e8bf807e902261 -- Name: bookshelf
[9/60] -- SynsetID: 02747177 -- InstanceID: 2ac7a4d88dfedcfef155d75bbf62b80  -- Name: ashcan,trash can,garbage can,wastebin,ash bin,ash-bin,ashbin,dustbin,trash barrel,tr

## Background Images

In [69]:
background_imgs_root = '/hdd/mliuzzolino/Places2/places365_standard/train'
background_img_paths = glob.glob(f'{background_imgs_root}/*/*')

## Load Images and Animate

In [70]:
def add_background(RGB_img, segmentation_img, background_img):
    mask = segmentation_img > 0
    RGB_img[np.where(~mask)] = background_img[np.where(~mask)]
    return RGB_img

In [71]:
def sample_background_img(background_img_paths, shape):
    random_background_path = np.random.choice(background_img_paths)
    background_img = imageio.imread(random_background_path)
    background_img = cv2.resize(background_img, shape[:2][::-1])
    return background_img

In [72]:
rendered_instance_ids

['render_output/synsetID_02691156/10af5de930178a161596c26b5af806fe',
 'render_output/synsetID_02691156/7f4b166fba71407490b1d6deb98feec6',
 'render_output/synsetID_02691156/3ecea45bfa541b8e4a4dd08ffc16eb81',
 'render_output/synsetID_02691156/b8e4e4994d4674cf2023ec956848b741',
 'render_output/synsetID_02691156/9d65814e1b252fb01636caafca838500',
 'render_output/synsetID_02691156/27c409ead0c4e34c9a6e43b878d5b335',
 'render_output/synsetID_02691156/420f3bb771e8e75ed878249aca2571f',
 'render_output/synsetID_02691156/8bde5a00c3caf9771d03b466c72ce41',
 'render_output/synsetID_02691156/8259a1fdcb9bca7526360e1e29a956c7',
 'render_output/synsetID_02691156/1cc3ebbadfe69e8011f5789deac2dcac',
 'render_output/synsetID_02691156/1d63eb2b1f78aa88acf77e718d93f3e1',
 'render_output/synsetID_02691156/48e477d5904bb7bb1ad94eee1d03defc',
 'render_output/synsetID_02691156/41c7470ce9ecb74b6f9423fcc87803f2',
 'render_output/synsetID_02691156/e8e1b765fdf5edfa14c19f41d007670e',
 'render_output/synsetID_02691156/55

In [73]:
rendered_synset_ids

['render_output/synsetID_02691156', 'render_output/synsetID_02958343']

In [76]:
_LOAD_LIMIT = 15

limit_reached = False

all_imgs = []
rendered_instance_ids = glob.glob(f"{_OUTPUT_ROOT}/*/*")
np.random.shuffle(rendered_instance_ids)
for rendered_instance_id in rendered_instance_ids:
    sys.stdout.write(f"\rProcessed model {len(all_imgs)+1}/{_LOAD_LIMIT}...")
    sys.stdout.flush()

    # Load paths
    rendered_img_paths = np.sort(glob.glob(f"{rendered_instance_id}/*RGBA_full.png"))
    semantic_seg_paths = [ele.replace('_full', '_semantic_segmentation') for ele in rendered_img_paths]

    # Load images
    rendered_imgs = [modules.utils.read_image(img_path) 
                         for img_path in rendered_img_paths]
    segmentation_imgs = [modules.utils.read_image(img_path)
                         for img_path in semantic_seg_paths]

    # Ensure correct number of view angles present
    if len(rendered_imgs) != _N_VIEW_ANGLES:
        print(f"\t**Error: Insufficient angles! Actual: {len(rendered_imgs)} - Expected: {_N_VIEW_ANGLES}")
        continue

    # Setup background image
    background_img = sample_background_img(background_img_paths, shape=rendered_imgs[0].shape)

    # Apply background image
    rendered_imgs = [add_background(rgb_img, seg_img, background_img) 
                         for rgb_img, seg_img in zip(rendered_imgs, segmentation_imgs)]

    # Join rgb image with semantic seg image
    rendered_imgs = [(rgb, mask) for rgb, mask in zip(rendered_imgs, segmentation_imgs)]

    # Combine images
    combined_imgs = modules.utils.combine_imgs(rendered_imgs)
    all_imgs.append(combined_imgs)

    # Check limit
    if len(all_imgs) >= _LOAD_LIMIT:
        print("\nLimit reached. Ending early.")
        limit_reached = True
        break

Processed model 15/15...
Limit reached. Ending early.


In [77]:
# Stack all images
stacked_imgs = modules.utils.stack_all_imgs(all_imgs, nrow=3)

In [78]:
savepath = 'test.gif'
animation = modules.utils.toAnimation(stacked_imgs, figsize=(12,12), interval=100, savepath=savepath, fps=15)

Building animator...
Generating animation object...
Saving to test.gif...
Fin.


In [79]:
if savepath.endswith('gif'):
    display(Image(filename=savepath))